In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [ ]:
!wget https://github.com/TobiasGlaubach/python-ml-turbofan/archive/master.zip

In [ ]:
!unzip master.zip


In [ ]:
!ls

In [ ]:
path='/home/spark/shared/python-ml-turbofan-master/CMAPSSData'
print(path)


In [ ]:
os.listdir(path)

In [ ]:
train_files= [filename for filename in os.listdir(path) if(filename.startswith('train') and filename.endswith('.txt'))]
                                                           

In [ ]:
train_files[0]

In [ ]:
test_files= [filename for filename in os.listdir(path) if(filename.startswith('test') and filename.endswith('.txt'))]

In [ ]:
print(train_files)
test_files

In [ ]:
!ls

In [ ]:
train_rdd=sc.textFile(path+'/'+train_files[0])

In [ ]:
train_rdd=train_rdd.map(lambda line:line.split(" "))

In [ ]:
train_rdd.count()

In [ ]:
test_rdd=sc.textFile(path+'/'+test_files[0])
test_rdd=test_rdd.map(lambda line:line.split(" "))

In [ ]:
test_rdd.count()

In [ ]:
header = ["id", "cycle", "setting1", "setting2", "setting3",
                    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8",
                    "s9", "s10", "s11", "s12", "s13", "s14", "s15", 
                    "s16","s17", "s18", "s19", "s20", "s21"]

In [ ]:
train_df=train_rdd.toDF(header)

In [ ]:
train_df.collect()

In [ ]:
test_df=test_rdd.toDF(header)

In [ ]:
test_df.show()

In [ ]:
import pandas as pd
train_pandas_df=train_df.toPandas()

In [ ]:
train_pandas_df.head()

In [ ]:
test_pandas_df=test_df.toPandas()
test_pandas_df.head()

In [ ]:
train_pandas_df.isnull().sum()


In [ ]:
train_df.na.drop()

In [ ]:
test_df.na.drop()

In [ ]:
train_pandas_df.dropna()

In [ ]:
df = train_df.withColumn('id', train_df['id'].cast(IntegerType()))

In [ ]:
df.collect()

In [ ]:
def set_type(df,intlist=[],doublelist=[]):
    if len(intlist)>0:
        for col in intlist:
            df=df.withColumn(col,df[col].cast(IntegerType()))
                             
    if len(doublelist)>0:
        for col in doublelist:
            df=df.withColumn(col,df[col].cast(DoubleType()))
    return df                            
                             

In [ ]:
train_df.columns

In [ ]:
int_list = ["id", "cycle"]
double_list= list(set(train_df.columns)-set(int_list) )
double_list

In [ ]:
train_df=set_type(train_df,int_list,double_list)
test_df=set_type(test_df,int_list,double_list)

In [ ]:
train_df.columns

In [ ]:
test_df.schema

In [ ]:
train_df.schema

In [ ]:
print ("For train data:")
train_df.groupBy('id').count().sort(F.col("count")).show(1)


In [ ]:
train_df.groupBy('id').count().sort(F.col("count")).show(1)

In [ ]:
train_df.groupBy('id').count().sort(F.col("count").desc()).show(1)

In [ ]:
test_df.groupBy('id').count().show()

In [ ]:
train_df.toPandas().count()

# 2020-02-04

In [ ]:
rul=train_df.groupBy('id').agg({"cycle":"max"}).sort(("id"))

In [ ]:
type(rul)

In [ ]:
rul=rul.toDF('id','max')

In [ ]:
rul.show()

In [ ]:
temp_df=train_df.join(rul,on=['id'],how='left').toPandas()

In [ ]:
temp_df.sort_values(by=["id"])

In [ ]:
temp_df['RUL']=temp_df['max']-temp_df['cycle']

In [ ]:
temp_df.sort_values(by=["id","cycle"])

In [54]:
def add_rul_labelling(df,id,time):
    rul=df.groupBy(id).agg({time:"max"}).sort('id')
    rul=rul.toDF('id','max')
    df=df.join(rul,on=['id'],how='left')
    df=df.withColumn("rul",df["max"]-df[time])
    return df

In [55]:
df=add_rul_labelling(train_df,"id","cycle")

In [56]:
df.toPandas()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,_27,_28,max,rul
0,31,1,-0.0009,-0.0000,100.0,518.67,642.32,1582.38,1393.03,14.62,...,0.03,392.0,2388.0,100.0,39.02,23.4287,None,None,234,233
1,31,2,-0.0005,-0.0003,100.0,518.67,642.49,1584.07,1394.01,14.62,...,0.03,393.0,2388.0,100.0,39.01,23.4805,None,None,234,232
2,31,3,0.0029,-0.0002,100.0,518.67,642.49,1585.07,1394.99,14.62,...,0.03,391.0,2388.0,100.0,39.05,23.3977,None,None,234,231
3,31,4,-0.0037,-0.0002,100.0,518.67,642.21,1572.40,1394.07,14.62,...,0.03,391.0,2388.0,100.0,39.02,23.3805,None,None,234,230
4,31,5,-0.0011,0.0005,100.0,518.67,641.84,1584.77,1396.82,14.62,...,0.03,393.0,2388.0,100.0,39.11,23.3663,None,None,234,229
5,31,6,-0.0003,-0.0003,100.0,518.67,642.48,1582.50,1398.27,14.62,...,0.03,391.0,2388.0,100.0,38.97,23.5206,None,None,234,228
6,31,7,-0.0031,0.0005,100.0,518.67,642.30,1584.57,1405.87,14.62,...,0.03,391.0,2388.0,100.0,39.10,23.3069,None,None,234,227
7,31,8,0.0032,-0.0002,100.0,518.67,642.32,1586.33,1401.08,14.62,...,0.03,391.0,2388.0,100.0,38.97,23.4163,None,None,234,226
8,31,9,0.0037,-0.0001,100.0,518.67,641.84,1585.09,1402.92,14.62,...,0.03,391.0,2388.0,100.0,38.90,23.4756,None,None,234,225
9,31,10,-0.0027,-0.0001,100.0,518.67,642.64,1595.07,1399.16,14.62,...,0.03,389.0,2388.0,100.0,39.12,23.4690,None,None,234,224
